# Flow ML Course

## TP 2 : Vertex AI  SDK Python

L'objectif de ce notebook est d'apprendre à utiliser les experiments de Flow ML.

Les Experiments permettent de logger les méta paramètres des architectures des modèles et leurs métriques. On peut regrouper les différents entrainements sous une experiment pour les comparer.


En savoir plus [Experiments]( https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments).

En savoir plus [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction).

## Initialisation

In [ ]:
PROJECT_ID = "projet-ia-448520" # "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

BUCKET_URI = "gs://cours1bucket" # f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

SERVICE_ACCOUNT = "837843744498-compute@developer.gserviceaccount.com" # "[your-service-account]"  # @param {type:"string"}

In [ ]:
import os
import uuid

import google.cloud.aiplatform as aiplatform

Pour ce TP nous allons utiliser le SDK python de Vertex AI. L'objet central de ce module est : aiplatform

Initialisons le :

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Créer une Experiment



- `aiplatform.init()` - Créer une experiment
- `aiplatform.start_run()` - Tracker un run spécifique de l'experiement

In [ ]:
# Specify a name for the experiment
your_name = ...
EXPERIMENT_NAME = f"supaero-vertex-course-1-2_{your_name}"


In [ ]:
# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

## Logger les parameters d'une experiment

Lors d'un entrainement, à un dataset spécifique et une architecture de modèle, plusieurs entrainement peuvent être fait pour tester les différentes configurations possibles :


- Traitement Données
    - Split Dataset
    - Sampling, Boosting, under/over sampling dataset
- Metaparameters
    - Profondeur et taille des layers par exemple
- Hyperparameters:
    - batch size
    - learning rate

On utilisera la méthode suivante pour stocker ces informations sous forme de key-value `log_params()`

In [ ]:
metaparams = {}
metaparams["units"] = 128
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

## Logger les métriques 

A la fin de l'entrainement, on peut logger les performances du modèle via :
- métriques d'évaluation
- hyperparameter search selection
- temps d'entrainement 
- early stop trigger

On utilisera la méthode suivante pour stocker ces informations sous forme de key-value `log_metrics()`

In [ ]:
metrics = {}
metrics["test_acc"] = 93.7
metrics["train_acc"] = 97.3
aiplatform.log_metrics(metrics)

## Extraire les résultats d'un Experiment

Pour terminer une expirement : utiliser `end_run()` 

Ensuite, vous pouvez récupérer la plupart des informations de l'experiment via : `get_experiment_df()` 

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

### A vous de jouer

Logger une nouvelle run sur la même experiement.

Changer certains paramètres dans le but d'améliorer un hypothétique entrainement.

Modifier l'accuracy pour éviter l'overfitting.


In [ ]:
aiplatform.start_run("run-2")

metaparams = {}
metaparams["units"] = ...  # changed the value
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = ...
hyperparams["batch_size"] = ...
hyperparams["learning_rate"] = ...
aiplatform.log_params(hyperparams)

metrics = {}
metrics["test_acc"] = ...
metrics["train_acc"] = ...
aiplatform.log_metrics(metrics)

## Comparer une Experimenet 

Explorer la pandas.DataFrame des résultats de l'experiment via  `get_experiment_df()` 

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

#### Supprimer l'Experiment


In [ ]:
exp = aiplatform.Experiment(EXPERIMENT_NAME)
try:
    exp.delete()
except Exception as e:
    print(e)

## Your Turn

Trouver la meilleur run de l'experiemnt supaero-vertex-course-1-1 à l'aide du SDK Vertex AI Python